In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")




In [3]:
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
weights = baseline.calculate("household_weight", period=2024)



In [4]:
baseline_aca_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=2026).sum()
baseline_aca_eligible/1e6

31.542351631973784

In [5]:
baseline_aca_enrollment = baseline.calculate("takes_up_aca_if_eligible", map_to="person", period=2026).sum()
baseline_aca_enrollment/1e6

225.39480517300126

In [6]:
period = 2025
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


205,150,194 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [7]:
period = 2026

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up_r = reformed.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc_r  = reformed.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt_r = reformed.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up_r == 1) & (aca_ptc_r > 0)

people_with_ptc_takeup_wtd_r = (mask.astype(float) * person_wt_r).sum()

print(f"{people_with_ptc_takeup_wtd_r:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


22,890,509 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [8]:
period = 2026
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


16,961,632 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [9]:
year = 2026
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
aca_baseline = baseline.calculate("aca_ptc", map_to="household", period=year)
rating_area = baseline.calculate("slcsp_rating_area", map_to="household", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)
aca_reform = reformed.calculate("aca_ptc", map_to="household", period=year)

In [10]:
# Create a DataFrame with the outputs
data = {
    "household_id": household_id,
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "aca_baseline": aca_baseline,
    "aca_reform": aca_reform,

   }


df_outputs = pd.DataFrame(data)
df_outputs[df_outputs['household_id'] == 4428]


,household_id,State,Married,Num_Dependents,Employment_Income,aca_baseline,aca_reform
459,4428,MA,1.0,4.0,52859.65625,0.0,0.0


In [11]:
# -------------------------------------------------------------
# 0️⃣  Make sure the CPS household weight is in the DataFrame
# -------------------------------------------------------------
# If you already stuffed it in earlier, skip this.
df_outputs["weight"] = aca_baseline.weights    # aligns by household_id

# -------------------------------------------------------------
# 1️⃣  Define a weight threshold for “reasonably representative”
# -------------------------------------------------------------
MIN_WT = 10_000          # ↖ try 5_000 if you want a looser cut

df_big = df_outputs[df_outputs["weight"] >= MIN_WT].copy()

# -------------------------------------------------------------
# 2️⃣  Net PTC change and (optionally) weighted national impact
# -------------------------------------------------------------
df_big["net_change"] = df_big["aca_reform"] - df_big["aca_baseline"]
df_big["wt_change"]  = df_big["net_change"] * df_big["weight"]  # national $ impact

# -------------------------------------------------------------
# 3️⃣  Biggest ↑ increases and ↓ decreases, LIMITED to big-weight HHs
# -------------------------------------------------------------
N = 10   # how many households to show in each direction

cols = ["household_id", "State", "weight", "net_change", "wt_change"]

top_increases = df_big.nlargest(N, "net_change")[cols]
top_decreases = df_big.nsmallest(N, "net_change")[cols]

print("Most positive net-income changes (PTC boosts):")
display(top_increases)

print("\nMost negative net-income changes (PTC cuts):")
display(top_decreases)


Most positive net-income changes (PTC boosts):


,household_id,State,weight,net_change,wt_change
3930,22572,WI,14746.737305,20787.404785,3.065464e+08
9715,63406,TX,313614.062500,20464.279297,6.417886e+09
7593,47863,FL,13494.670898,20171.796875,2.722118e+08
12472,83838,CA,42713.929688,18523.296875,7.912028e+08
11078,74935,UT,19624.578125,14949.250000,2.933727e+08
7287,46034,FL,191966.390625,13774.429688,2.644228e+09
1718,11792,NJ,22938.029297,9646.639648,2.212749e+08
5058,29932,KS,10356.357422,9572.796875,9.913931e+07
3335,19714,IL,12393.625000,7891.378906,9.780279e+07
4053,23036,WI,20458.789062,7672.568359,1.569715e+08



Most negative net-income changes (PTC cuts):


,household_id,State,weight,net_change,wt_change
0,12,ME,28690.535156,0.0,0.0
1,21,ME,10654.151367,0.0,0.0
8,73,ME,10017.615234,0.0,0.0
10,79,ME,21640.277344,0.0,0.0
20,134,ME,21905.371094,0.0,0.0
24,194,ME,14491.523438,0.0,0.0
26,206,ME,23982.746094,0.0,0.0
28,261,ME,14972.551758,0.0,0.0
41,356,ME,13415.000000,0.0,0.0
45,407,ME,10767.994141,0.0,0.0


In [12]:
df_outputs[df_outputs['household_id'] == 4428]


,household_id,State,Married,Num_Dependents,Employment_Income,aca_baseline,aca_reform,weight
459,4428,MA,1.0,4.0,52859.65625,0.0,0.0,4397.432129


In [13]:
# 0. Make sure net_change exists
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# 1. Flag households with any change
mask = df_outputs["net_change"] != 0          # True for ↑ or ↓

# 2. Weighted mean among those households
avg_net_change = (
    (df_outputs.loc[mask, "net_change"] * df_outputs.loc[mask, "weight"]).sum()
    / df_outputs.loc[mask, "weight"].sum()
)

print(f"Average weighted PTC change among households with any change: "
      f"${avg_net_change:,.2f}")


Average weighted PTC change among households with any change: $2,730.13


In [14]:
# ------------------------------------------------------------------
# 0.  Ensure supporting columns exist
# ------------------------------------------------------------------
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# ------------------------------------------------------------------
# 1.  Keep only households with a PTC in *both* scenarios
# ------------------------------------------------------------------
mask_both_ptc = (df_outputs["aca_baseline"] > 0) & (df_outputs["aca_reform"] > 0)
df_dual_ptc   = df_outputs[mask_both_ptc]

# ------------------------------------------------------------------
# 2.  Weighted average of the net change (household perspective)
# ------------------------------------------------------------------
avg_net_change_dual_hh = (
    (df_dual_ptc["net_change"] * df_dual_ptc["weight"]).sum()
    / df_dual_ptc["weight"].sum()
)

print(f"Average weighted PTC change among households with a PTC in both "
      f"baseline and reform: ${avg_net_change_dual_hh:,.2f}")


Average weighted PTC change among households with a PTC in both baseline and reform: $2,239.05


In [15]:
# ------------------------------------------------------------------
# 0.  Ensure supporting columns exist (already done above)
# ------------------------------------------------------------------
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# ------------------------------------------------------------------
# 1.  Keep only households that *gain* a PTC (reform > 0, baseline == 0)
# ------------------------------------------------------------------
mask_reform_only = (df_outputs["aca_baseline"] == 0) & (df_outputs["aca_reform"] > 0)
df_reform_only   = df_outputs[mask_reform_only]

# ------------------------------------------------------------------
# 2.  Weighted average of the net change (household perspective)
# ------------------------------------------------------------------
avg_net_change_reform_only_hh = (
    (df_reform_only["net_change"] * df_reform_only["weight"]).sum()
    / df_reform_only["weight"].sum()
)

print(f"Average weighted PTC change among households that newly receive a PTC "
      f"under the reform: ${avg_net_change_reform_only_hh:,.2f}")


Average weighted PTC change among households that newly receive a PTC under the reform: $5,302.77


In [16]:
import numpy as np
from policyengine_us import Simulation

# -------------------------------
# 1. Pull household-level results
# -------------------------------
# ACA PTC (baseline and reform)
ptc_base   = baseline.calculate("aca_ptc", map_to="household", period=2026)
ptc_reform = reformed.calculate("aca_ptc",   map_to="household", period=2026)

# Household weights (same for both sims)
hh_wt      = baseline.calculate("household_weight", map_to="household", period=2026)

# -------------------------------
# 2. Weighted sum of the change
# -------------------------------
weighted_total_change = ptc_reform - ptc_base

# Optional: average change per household
weighted_total_change.sum()/1e9

32.76222811087988

In [17]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ------------------------------------------------------------------
# Brand hex codes (one-to-one with style.colors)
# ------------------------------------------------------------------
COLOR_BLUE               = "#2C6496"   # style.colors.BLUE  / BLUE_PRIMARY
COLOR_BLUE_LIGHT         = "#D8E6F3"   # style.colors.BLUE_LIGHT / BLUE_95
COLOR_LIGHT_GRAY         = "#F2F2F2"   # style.colors.LIGHT_GRAY
COLOR_MEDIUM_LIGHT_GRAY  = "#BDBDBD"   # style.colors.MEDIUM_LIGHT_GRAY
COLOR_DARK_GRAY          = "#616161"   # style.colors.DARK_GRAY

# ––– choose colours for positive vs. negative average bars –––
POS_COLOR = COLOR_BLUE
NEG_COLOR = COLOR_DARK_GRAY

# ------------------------------------------------------------------
# 1.  Pull baseline / reform net income + weights
# ------------------------------------------------------------------
net_base   = baseline.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
net_reform = reformed.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
weights    = baseline.calculate(
    "household_weight", map_to="household", period=2026
)

df = pd.DataFrame({
    "net_base": net_base,
    "delta":    net_reform - net_base,
    "weight":   weights,
})

# ------------------------------------------------------------------
# 2.  Weighted decile edges (baseline ranking)
# ------------------------------------------------------------------
def wquantile(values, qs, w):
    srt = np.argsort(values)
    values, w = values[srt], w[srt]
    cum_w = np.cumsum(w) / np.sum(w)
    return np.interp(qs, cum_w, values)

edges = wquantile(df["net_base"].values,
                  np.linspace(0, 1, 11), df["weight"].values)

df["decile"] = pd.cut(df["net_base"],
                      bins=edges,
                      labels=np.arange(1, 11),
                      include_lowest=True)

# ------------------------------------------------------------------
# 3.  Weighted average Δnet-income by decile
# ------------------------------------------------------------------
decile_avg = (
    df.groupby("decile")
      .apply(lambda g: np.average(g["delta"], weights=g["weight"]))
      .reset_index(name="avg_change")
)

# ------------------------------------------------------------------
# 4.  Use brand colours: blue if gain, dark-gray if loss
# ------------------------------------------------------------------
bar_colors = [
    POS_COLOR if v >= 0 else NEG_COLOR
    for v in decile_avg["avg_change"]
]

# ------------------------------------------------------------------
# 5.  Plot
# ------------------------------------------------------------------
fig = go.Figure(
    data=[
        go.Bar(
            x=decile_avg["decile"].astype(int),
            y=decile_avg["avg_change"],
            marker_color=bar_colors,
            text=decile_avg["avg_change"].apply(lambda v: f"${v:,.0f}"),
            textposition="inside",
        )
    ],
    layout=dict(
        title="Impact of Extending IRA PTC Expansion by Income Decile – 2026",
        xaxis_title="Income Decile",
        yaxis_title="Average change in household net income ($)",
        showlegend=False,
    )
)
fig.add_hline(y=0, line_width=1, line_color="black")
fig.show()
fig.update_xaxes(dtick=1)        # show 1-10 instead of only the evens


In [18]:
# Let's analyze the households affected by the ACA reform
import pandas as pd
import numpy as np

# First, let's look at households that gain PTC under reform but had none in baseline
gained_ptc = df_outputs[(df_outputs['aca_baseline'] == 0) & (df_outputs['aca_reform'] > 0)]

print(f"Number of households gaining PTC under reform: {len(gained_ptc)}")
print(f"Weighted count: {gained_ptc['weight'].sum():,.0f}")
print(f"\nAverage reform PTC for these households: ${gained_ptc['aca_reform'].mean():,.2f}")
print(f"Weighted average reform PTC: ${(gained_ptc['aca_reform'] * gained_ptc['weight']).sum() / gained_ptc['weight'].sum():,.2f}")

Number of households gaining PTC under reform: 761
Weighted count: 1,923,488

Average reform PTC for these households: $5,751.24
Weighted average reform PTC: $5,302.77


In [19]:
# Let's look at income distribution of households gaining PTC
import matplotlib.pyplot as plt

# Add income deciles to the gained_ptc dataframe
gained_ptc_with_income = gained_ptc.copy()

# Calculate weighted income percentiles for context
income_percentiles = np.percentile(df_outputs['Employment_Income'], [25, 50, 75, 90, 95])
print("Income percentiles across all households:")
for i, pct in enumerate([25, 50, 75, 90, 95]):
    print(f"  {pct}th percentile: ${income_percentiles[i]:,.0f}")

# Show income distribution of households gaining PTC
print("\nIncome distribution of households GAINING PTC under reform:")
print(gained_ptc_with_income['Employment_Income'].describe())

# Show top 10 households by PTC gain amount
print("\nTop 10 households by PTC gain (sorted by reform PTC amount):")
top_gainers = gained_ptc_with_income.nlargest(10, 'aca_reform')[['household_id', 'State', 'Employment_Income', 'aca_reform', 'Married', 'Num_Dependents', 'weight']]
top_gainers

ModuleNotFoundError: No module named 'matplotlib'

In [20]:
# Let's continue without matplotlib
# Add income deciles to the gained_ptc dataframe
gained_ptc_with_income = gained_ptc.copy()

# Calculate weighted income percentiles for context
income_percentiles = np.percentile(df_outputs['Employment_Income'], [25, 50, 75, 90, 95])
print("Income percentiles across all households:")
for i, pct in enumerate([25, 50, 75, 90, 95]):
    print(f"  {pct}th percentile: ${income_percentiles[i]:,.0f}")

print("\n" + "="*60)
print("Income distribution of households GAINING PTC under reform:")
print("="*60)
print(gained_ptc_with_income['Employment_Income'].describe())

print("\n" + "="*60)
print("Top 10 households by PTC gain (sorted by reform PTC amount):")
print("="*60)
top_gainers = gained_ptc_with_income.nlargest(10, 'aca_reform')[['household_id', 'State', 'Employment_Income', 'aca_reform', 'Married', 'Num_Dependents', 'weight']]
display(top_gainers)

Income percentiles across all households:
  25th percentile: $8,776
  50th percentile: $59,168
  75th percentile: $121,807
  90th percentile: $205,693
  95th percentile: $287,212

Income distribution of households GAINING PTC under reform:
count       761.000000
mean     113532.674681
std       67906.364616
min           0.000000
25%       68947.382812
50%      103907.218750
75%      150131.617188
max      467888.655273
Name: Employment_Income, dtype: float64

Top 10 households by PTC gain (sorted by reform PTC amount):


,household_id,State,Employment_Income,aca_reform,Married,Num_Dependents,weight
10241,69304,WY,172368.441406,27582.343750,1.0,3.0,6615.325195
4600,25635,MO,37921.058594,27411.421875,1.0,0.0,4003.946045
17827,136329,FL,96147.054688,27191.490234,1.0,1.0,0.052937
9197,59697,OK,68947.382812,26790.708984,1.0,0.0,1484.846436
4610,25669,MO,0.000000,26257.669922,1.0,0.0,1353.809692
4565,25484,MO,120657.902344,25160.201172,1.0,0.0,4692.614746
20880,170145,CA,97803.962158,25064.998047,1.0,0.0,1.485237
21249,173580,CA,107563.710938,24998.007812,1.0,0.0,0.000211
21284,173817,CA,120305.722656,24842.128906,1.0,0.0,0.003597
62,548,ME,0.000000,24623.031250,1.0,0.0,1530.439819


In [20]:
# Let's continue without matplotlib
# Add income deciles to the gained_ptc dataframe
gained_ptc_with_income = gained_ptc.copy()

# Calculate weighted income percentiles for context
income_percentiles = np.percentile(df_outputs['Employment_Income'], [25, 50, 75, 90, 95])
print("Income percentiles across all households:")
for i, pct in enumerate([25, 50, 75, 90, 95]):
    print(f"  {pct}th percentile: ${income_percentiles[i]:,.0f}")

print("\n" + "="*60)
print("Income distribution of households GAINING PTC under reform:")
print("="*60)
print(gained_ptc_with_income['Employment_Income'].describe())

print("\n" + "="*60)
print("Top 10 households by PTC gain (sorted by reform PTC amount):")
print("="*60)
top_gainers = gained_ptc_with_income.nlargest(10, 'aca_reform')[['household_id', 'State', 'Employment_Income', 'aca_reform', 'Married', 'Num_Dependents', 'weight']]
top_gainers

Income percentiles across all households:
  25th percentile: $8,776
  50th percentile: $59,168
  75th percentile: $121,807
  90th percentile: $205,693
  95th percentile: $287,212

Income distribution of households GAINING PTC under reform:
count       761.000000
mean     113532.674681
std       67906.364616
min           0.000000
25%       68947.382812
50%      103907.218750
75%      150131.617188
max      467888.655273
Name: Employment_Income, dtype: float64

Top 10 households by PTC gain (sorted by reform PTC amount):


,household_id,State,Employment_Income,aca_reform,Married,Num_Dependents,weight
10241,69304,WY,172368.441406,27582.343750,1.0,3.0,6615.325195
4600,25635,MO,37921.058594,27411.421875,1.0,0.0,4003.946045
17827,136329,FL,96147.054688,27191.490234,1.0,1.0,0.052937
9197,59697,OK,68947.382812,26790.708984,1.0,0.0,1484.846436
4610,25669,MO,0.000000,26257.669922,1.0,0.0,1353.809692
4565,25484,MO,120657.902344,25160.201172,1.0,0.0,4692.614746
20880,170145,CA,97803.962158,25064.998047,1.0,0.0,1.485237
21249,173580,CA,107563.710938,24998.007812,1.0,0.0,0.000211
21284,173817,CA,120305.722656,24842.128906,1.0,0.0,0.003597
62,548,ME,0.000000,24623.031250,1.0,0.0,1530.439819


In [21]:
# Now let's look at households losing PTC or seeing reduced PTC
lost_or_reduced = df_outputs[(df_outputs['aca_baseline'] > 0) & (df_outputs['net_change'] < 0)]

print("Households LOSING or SEEING REDUCED PTC:")
print(f"Number of households: {len(lost_or_reduced)}")
print(f"Weighted count: {lost_or_reduced['weight'].sum():,.0f}")
print(f"\nAverage baseline PTC: ${lost_or_reduced['aca_baseline'].mean():,.2f}")
print(f"Average reform PTC: ${lost_or_reduced['aca_reform'].mean():,.2f}")
print(f"Average loss: ${lost_or_reduced['net_change'].mean():,.2f}")

# Income distribution
print("\nIncome distribution of households losing PTC benefits:")
print(lost_or_reduced['Employment_Income'].describe())

# Top losers
print("\nTop 10 households by PTC loss:")
top_losers = lost_or_reduced.nsmallest(10, 'net_change')[['household_id', 'State', 'Employment_Income', 'aca_baseline', 'aca_reform', 'net_change', 'weight']]
top_losers

Households LOSING or SEEING REDUCED PTC:
Number of households: 0
Weighted count: 0

Average baseline PTC: $nan
Average reform PTC: $nan
Average loss: $nan

Income distribution of households losing PTC benefits:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Employment_Income, dtype: float64

Top 10 households by PTC loss:


,household_id,State,Employment_Income,aca_baseline,aca_reform,net_change,weight


In [22]:
# Interesting - no households lose PTC! Let's look at those who keep their PTC but see changes
kept_ptc = df_outputs[(df_outputs['aca_baseline'] > 0) & (df_outputs['aca_reform'] > 0)]

print("Households with PTC in BOTH baseline and reform:")
print(f"Number of households: {len(kept_ptc)}")
print(f"Weighted count: {kept_ptc['weight'].sum():,.0f}")
print(f"\nAverage baseline PTC: ${kept_ptc['aca_baseline'].mean():,.2f}")
print(f"Average reform PTC: ${kept_ptc['aca_reform'].mean():,.2f}")
print(f"Average change: ${kept_ptc['net_change'].mean():,.2f}")

# Show distribution of changes
print("\nDistribution of PTC changes for households with PTC in both scenarios:")
print(kept_ptc['net_change'].describe())

# Households with biggest increases among those who already had PTC
print("\nTop 10 PTC increases among households who already had PTC:")
top_increases = kept_ptc.nlargest(10, 'net_change')[['household_id', 'State', 'Employment_Income', 'aca_baseline', 'aca_reform', 'net_change', 'weight']]
top_increases

Households with PTC in BOTH baseline and reform:
Number of households: 2437
Weighted count: 10,076,767

Average baseline PTC: $7,909.13
Average reform PTC: $9,880.77
Average change: $1,971.64

Distribution of PTC changes for households with PTC in both scenarios:
count     2437.000000
mean      1971.639511
std       1562.269100
min        433.568359
25%       1304.000244
50%       1613.736328
75%       2232.999512
max      29610.569824
Name: net_change, dtype: float64

Top 10 PTC increases among households who already had PTC:


,household_id,State,Employment_Income,aca_baseline,aca_reform,net_change,weight
13395,95388,CT,230258.468750,2271.705078,31882.274902,29610.569824,0.000197
20794,169452,CA,131771.823242,6317.419434,28958.584473,22641.165039,0.003905
12985,88926,HI,119508.789062,3365.458740,25774.730469,22409.271729,1788.657227
3930,22572,WI,91929.835449,5390.636230,26178.041016,20787.404785,14746.737305
9715,63406,TX,56307.025391,3461.937988,23926.217285,20464.279297,313614.062500
19463,153886,TX,205251.378052,2576.328125,22568.280273,19991.952148,0.000294
20786,169392,CA,254882.910156,5029.621582,23572.651611,18543.030029,0.001491
12778,85992,CA,264716.572266,2762.978271,17948.256836,15185.278564,2075.729736
20062,162791,AZ,39974.335938,12875.888672,26409.888672,13534.000000,0.000893
17712,135165,FL,196369.458984,15325.860840,28293.105469,12967.244629,0.056075


In [23]:
# Let's calculate approximate FPL levels for households to understand where they fall
# 2026 FPL estimates (rough approximations based on current trends)
fpl_2026 = {
    1: 15570,   # Single person
    2: 21130,   # Couple
    3: 26650,   # Family of 3
    4: 32200,   # Family of 4
    5: 37750,   # Family of 5
    6: 43300,   # Family of 6
    7: 48850,   # Family of 7
    8: 54400,   # Family of 8
}

# Calculate household size and FPL ratio
df_outputs['household_size'] = 2 + df_outputs['Num_Dependents']  # Assuming married couples or singles with deps
df_outputs['household_size'] = df_outputs.apply(
    lambda row: (1 + row['Married'] + row['Num_Dependents']) if not pd.isna(row['Married']) else 1,
    axis=1
)

# Map FPL based on household size
df_outputs['fpl_threshold'] = df_outputs['household_size'].map(lambda x: fpl_2026.get(min(int(x), 8), 54400))
df_outputs['fpl_ratio'] = (df_outputs['Employment_Income'] / df_outputs['fpl_threshold']) * 100

# Now let's analyze the cliff effect around 400% FPL
print("="*70)
print("ANALYSIS OF THE 400% FPL CLIFF EFFECT")
print("="*70)

# Households just below and above 400% FPL
near_cliff = df_outputs[(df_outputs['fpl_ratio'] >= 350) & (df_outputs['fpl_ratio'] <= 450)]
print(f"\nHouseholds between 350-450% FPL: {len(near_cliff)}")
print(f"Weighted count: {near_cliff['weight'].sum():,.0f}")

# Split by those above and below 400% FPL
below_400 = near_cliff[near_cliff['fpl_ratio'] <= 400]
above_400 = near_cliff[near_cliff['fpl_ratio'] > 400]

print(f"\nBelow 400% FPL (350-400%): {len(below_400)} households")
print(f"  Average baseline PTC: ${below_400['aca_baseline'].mean():,.2f}")
print(f"  Average reform PTC: ${below_400['aca_reform'].mean():,.2f}")
print(f"  Average change: ${below_400['net_change'].mean():,.2f}")

print(f"\nAbove 400% FPL (400-450%): {len(above_400)} households")
print(f"  Average baseline PTC: ${above_400['aca_baseline'].mean():,.2f}")
print(f"  Average reform PTC: ${above_400['aca_reform'].mean():,.2f}")
print(f"  Average change: ${above_400['net_change'].mean():,.2f}")

# Show some examples
print("\n" + "="*70)
print("EXAMPLE HOUSEHOLDS AT THE CLIFF (395-405% FPL):")
print("="*70)
cliff_examples = df_outputs[(df_outputs['fpl_ratio'] >= 395) & (df_outputs['fpl_ratio'] <= 405)]
cliff_examples_display = cliff_examples[['household_id', 'State', 'Employment_Income', 'fpl_ratio', 
                                         'aca_baseline', 'aca_reform', 'net_change', 'weight']].head(10)
cliff_examples_display

ANALYSIS OF THE 400% FPL CLIFF EFFECT

Households between 350-450% FPL: 2108
Weighted count: 12,872,038

Below 400% FPL (350-400%): 1094 households
  Average baseline PTC: $1,367.79
  Average reform PTC: $1,867.28
  Average change: $499.49

Above 400% FPL (400-450%): 1014 households
  Average baseline PTC: $660.53
  Average reform PTC: $1,449.85
  Average change: $789.33

EXAMPLE HOUSEHOLDS AT THE CLIFF (395-405% FPL):


,household_id,State,Employment_Income,fpl_ratio,aca_baseline,aca_reform,net_change,weight
11,91,ME,85344.212891,403.900676,0.000000,0.000,0.000000,1078.256592
57,495,ME,105351.593262,395.315547,0.000000,0.000,0.000000,9808.822266
247,2971,VT,85035.097656,402.437755,0.000000,0.000,0.000000,973.623474
371,3945,MA,106868.429688,401.007241,0.000000,0.000,0.000000,53990.785156
497,4662,MA,62052.640625,398.539760,0.000000,0.000,0.000000,467.895142
600,5462,RI,106868.433594,401.007256,0.000000,0.000,0.000000,3926.910889
686,6168,CT,62052.640625,398.539760,0.000000,0.000,0.000000,106855.679688
846,6872,NY,85035.101562,402.437774,0.000000,0.000,0.000000,241672.859375
1165,8843,NY,85264.925781,403.525441,17897.585938,21417.375,3519.789062,1036.865601
1446,10425,NJ,129850.888672,403.263629,0.000000,0.000,0.000000,227.340012


In [24]:
# Let's look more specifically at the income deciles to see where the cliff effect shows up
# Calculate income deciles
df_outputs['income_decile'] = pd.qcut(df_outputs['Employment_Income'], 10, labels=range(1, 11), duplicates='drop')

# Group by decile and show the effect
decile_analysis = df_outputs.groupby('income_decile').agg({
    'Employment_Income': ['min', 'max', 'mean'],
    'fpl_ratio': 'mean',
    'aca_baseline': 'mean',
    'aca_reform': 'mean',
    'net_change': 'mean',
    'weight': 'sum'
}).round(2)

print("="*70)
print("PTC EFFECTS BY INCOME DECILE")
print("="*70)
print("\nIncome ranges and average PTC changes by decile:")
decile_analysis

ValueError: Bin labels must be one fewer than the number of bin edges

In [25]:
# Fix the decile calculation
# Calculate income deciles without explicit labels to avoid the error
df_outputs['income_decile'] = pd.qcut(df_outputs['Employment_Income'], 10, duplicates='drop')

# Get unique deciles and sort them
deciles = sorted(df_outputs['income_decile'].unique())

# Create a mapping to simpler labels
decile_map = {d: i+1 for i, d in enumerate(deciles)}
df_outputs['decile_num'] = df_outputs['income_decile'].map(decile_map)

# Group by decile and show the effect
decile_analysis = df_outputs.groupby('decile_num').agg({
    'Employment_Income': ['min', 'max', 'mean'],
    'fpl_ratio': 'mean',
    'aca_baseline': 'mean',
    'aca_reform': 'mean',
    'net_change': 'mean',
    'weight': 'sum'
}).round(2)

print("="*70)
print("PTC EFFECTS BY INCOME DECILE")
print("="*70)
print("\nIncome ranges and average PTC changes by decile:")
decile_analysis

PTC EFFECTS BY INCOME DECILE

Income ranges and average PTC changes by decile:


Employment_Income                        fpl_ratio aca_baseline  \
                         min         max       mean      mean         mean   
decile_num                                                                   
1                       0.00    22082.70    3081.23     16.50       481.01   
2                   22096.56    40219.30   31811.30    167.52      1638.77   
3                   40220.45    59168.34   50102.24    254.53      1575.92   
4                   59179.83    80438.61   69904.57    343.95      1437.25   
5                   80438.61   106574.53   93253.97    434.80      1303.70   
6                  106597.34   142491.25  122908.07    554.42       816.63   
7                  142491.25   205693.01  169652.75    727.25       489.02   
8                  205693.02  3305428.97  382158.51   1548.38       210.36   

           aca_reform net_change       weight  
                 mean       mean          sum  
decile_num                                     
1              618.01     136.99  56060974.39  
2             1989.05     350.27  18267145.22  
3             2062.44     486.52  13547056.92  
4             2029.91     592.66  12499143.93  
5             2060.30     756.60  11057517.34  
6             1518.36     701.72  13853349.38  
7             1177.16     688.14  10689611.79  
8              482.04     271.67  11524346.53

In [ ]:
# Let's look at where in the data the 9th decile falls (the one from the chart)
# Since we only have 8 groups due to duplicates being dropped, let's recalculate properly

# First, let's understand the actual income distribution better
print("="*70)
print("UNDERSTANDING THE 9TH DECILE CONCENTRATION")
print("="*70)

# Get percentiles to understand income distribution
percentiles = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99]
income_pcts = np.percentile(df_outputs['Employment_Income'], percentiles)

print("\nIncome distribution percentiles:")
for p, val in zip(percentiles, income_pcts):
    print(f"  {p}th percentile: ${val:,.0f}")

# The 9th decile should be roughly between 80th and 90th percentile
ninth_decile = df_outputs[(df_outputs['Employment_Income'] >= income_pcts[7]) & 
                          (df_outputs['Employment_Income'] < income_pcts[8])]

print(f"\n9th Decile (80-90th percentile):")
print(f"  Income range: ${income_pcts[7]:,.0f} - ${income_pcts[8]:,.0f}")
print(f"  Number of households: {len(ninth_decile)}")
print(f"  Weighted count: {ninth_decile['weight'].sum():,.0f}")
print(f"  Average FPL ratio: {ninth_decile['fpl_ratio'].mean():.1f}%")
print(f"  Average baseline PTC: ${ninth_decile['aca_baseline'].mean():,.2f}")
print(f"  Average reform PTC: ${ninth_decile['aca_reform'].mean():,.2f}")
print(f"  Average change: ${ninth_decile['net_change'].mean():,.2f}")

# Now let's see WHO specifically gains in the 9th decile
ninth_decile_gainers = ninth_decile[ninth_decile['net_change'] > 100]  # Gains more than $100

print(f"\nHouseholds in 9th decile with gains > $100:")
print(f"  Count: {len(ninth_decile_gainers)}")
print(f"  Average income: ${ninth_decile_gainers['Employment_Income'].mean():,.0f}")
print(f"  Average FPL ratio: {ninth_decile_gainers['fpl_ratio'].mean():.1f}%")
print(f"  Average gain: ${ninth_decile_gainers['net_change'].mean():,.2f}")

# Look at specific examples
print("\nExample households in 9th decile with large gains:")
examples = ninth_decile_gainers.nlargest(5, 'net_change')[
    ['household_id', 'State', 'Employment_Income', 'fpl_ratio', 
     'aca_baseline', 'aca_reform', 'net_change', 'Married', 'Num_Dependents']
]
examples

In [26]:
# Check if the dataframe exists and has the needed columns
print("Checking available variables:")
print(f"df_outputs exists: {'df_outputs' in locals()}")
if 'df_outputs' in locals():
    print(f"df_outputs shape: {df_outputs.shape}")
    print(f"Columns: {list(df_outputs.columns)}")

Checking available variables:
df_outputs exists: True
df_outputs shape: (21607, 14)
Columns: ['household_id', 'State', 'Married', 'Num_Dependents', 'Employment_Income', 'aca_baseline', 'aca_reform', 'weight', 'net_change', 'household_size', 'fpl_threshold', 'fpl_ratio', 'income_decile', 'decile_num']


In [ ]:
# Now let's analyze ESI coverage and zero income - CORRECTED
print("="*70)
print("ANALYZING ESI COVERAGE AND ZERO INCOME")
print("="*70)

# Get ESI status and employment income at person level
has_esi = baseline.calculate("has_esi", map_to="person", period=2026)
person_income = baseline.calculate("employment_income", map_to="person", period=2026)

# Get person weights directly (not through calculate)
person_weight = has_esi.weights

# Create masks for our conditions
has_esi_mask = (has_esi == 1)
zero_income_mask = (person_income == 0)
both_mask = has_esi_mask & zero_income_mask

# Calculate weighted counts
total_with_esi = (has_esi_mask * person_weight).sum()
total_with_zero_income = (zero_income_mask * person_weight).sum()
total_with_both = (both_mask * person_weight).sum()

print(f"\nTotal people with ESI: {total_with_esi:,.0f}")
print(f"Total people with zero employment income: {total_with_zero_income:,.0f}")
print(f"Total people with BOTH ESI and zero income: {total_with_both:,.0f}")

print(f"\nPercentage of ESI holders with zero income: {(total_with_both/total_with_esi)*100:.1f}%")
print(f"Percentage of zero-income people with ESI: {(total_with_both/total_with_zero_income)*100:.1f}%")

# Let's also break this down by age groups to understand better
age = baseline.calculate("age", map_to="person", period=2026)

# Create age groups
child_mask = (age < 18)
working_age_mask = (age >= 18) & (age < 65)
senior_mask = (age >= 65)

print("\n" + "="*70)
print("BREAKDOWN BY AGE GROUP")
print("="*70)

for age_group, age_mask, label in [
    ("Children (< 18)", child_mask, "child"),
    ("Working Age (18-64)", working_age_mask, "working"),
    ("Seniors (65+)", senior_mask, "senior")
]:
    group_esi_zero_income = has_esi_mask & zero_income_mask & age_mask
    group_count = (group_esi_zero_income * person_weight).sum()
    group_esi = (has_esi_mask & age_mask * person_weight).sum()
    
    print(f"\n{age_group}:")
    print(f"  With ESI and zero income: {group_count:,.0f}")
    print(f"  Total with ESI: {group_esi:,.0f}")
    if group_esi > 0:
        print(f"  Percentage: {(group_count/group_esi)*100:.1f}%")